    Τμήμα Πληροφορικής και Τηλεπικοινωνιών - Άρτα 
    Πανεπιστήμιο Ιωαννίνων 

    Γκόγκος Χρήστος 
    http://chgogos.github.io/
    Εαρινό εξάμηνο 2020-2021

# PLY

To PLY αποτελεί υλοποίηση σε Python των λογισμικών υποβοήθησης κατασκευής λεκτικών και συντακτικών αναλυτών lex και yacc.

<https://www.dabeaz.com/ply/ply.html>

## Δημιουργία απλού υπολογιστή πράξεων με το PLY

Ο υπολογιστής θα μπορεί να κάνει πράξεις (πρόσθεση αφαίρεση, πολλαπλασιασμό και διαίρεση) με ακέραιους και πραγματικούς αριθμούς. Θα υπάρχει η δυνατότητα χρήσης μεταβλητών. 

Τροποποίηση από το: https://www.youtube.com/playlist?list=PLBOh8f9FoHHg7Ed_4yKhIbq4lIJAlonn8

In [2]:
# Εγκατάσταση PLY

! pip install ply

In [3]:
import ply.lex as lex
import ply.yacc as yacc
import sys

### Λεκτικός αναλυτής

Ο λεκτικός αναλυτής εντοπίζει όλες τις λεκτικές μονάδες (tokens) της εισόδου

In [4]:
# Κώδικας δημιουργίας λεκτικού αναλυτή (lexer)

# Λίστα με ονόματα tokens
tokens = [
    'INT', 
    'FLOAT',
    'ID',
    'PLUS',
    'MINUS',
    'DIVIDE',
    'MULTIPLY',
    'EQUALS'
]

# Κανόνες κανονικών εκφράσεων για απλά tokens
t_PLUS = r'\+'
t_MINUS = r'\-'
t_MULTIPLY = r'\*'
t_DIVIDE = r'\/'
t_EQUALS = r'\='

# χαρακτήρες που θα πρέπει να αγνοούνται (διαστήματα και tabs)
t_ignore = r' \t'

# κανόνας που θα εκτελείται όταν θα αναγνωρίζεται ένας πραγματικός αριθμός
def t_FLOAT(t):
    r'\d+\.\d+'
    t.value = float(t.value)
    return t

# κανόνας που θα εκτελείται όταν θα αναγνωρίζεται ένας ακέραιος αριθμός
def t_INT(t):
    r'\d+'
    t.value = int(t.value)
    return t

# κανόνας που θα εκτελείται όταν θα αναγνωρίζεται ένα αναγνωριστικό
def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = 'ID'
    return t

# κανόνας χειρισμού λαθών
def t_error(t):
    print("Illegal character {t.value[0]}")
    t.lexer.skip(1)

# κανόνας για παρακολούθηση των γραμμών εντολών του προγράμματος
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

In [5]:
# "κόλπο" για να λειτουργήσει το PLY σε περιβάλλον notebook (https://stackoverflow.com/questions/36393114/using-ply-with-ipython-jupyter)
__file__ = "33-PLY.ipynb" 

# δημιουργία του lexer
lexer = lex.lex()

# εκτέλεση του lexer για μια έκφραση
lexer.input("x = 1 + 2")
while True:
    tok = lexer.token()
    if not tok:
        break
    print(tok)

LexToken(ID,'x',1,0)
LexToken(EQUALS,'=',1,2)
LexToken(INT,1,1,4)
LexToken(PLUS,'+',1,6)
LexToken(INT,2,1,8)


In [6]:
lexer = lex.lex()

lexer.input("x = 1 + 2")
for tok in lexer:
    print(f"{tok} -> {tok.lineno}, {tok.lexpos}, {tok.type}, {tok.value}")

LexToken(ID,'x',1,0) -> 1, 0, ID, x
LexToken(EQUALS,'=',1,2) -> 1, 2, EQUALS, =
LexToken(INT,1,1,4) -> 1, 4, INT, 1
LexToken(PLUS,'+',1,6) -> 1, 6, PLUS, +
LexToken(INT,2,1,8) -> 1, 8, INT, 2


### Συντακτικός αναλυτής (1/2)

Η γραμματική είναι η ακόλουθη

```
calc : expression
     | id_assign
     | empty

expression : expression MULTIPLY expression
           | expression DIVIDE expression
           | expression PLUS expression
           | expression MINUS expression
           | ID
           | INT 
           | FLOAT

id_assign : ID EQUALS expression

empty : 

```

Ο ακόλουθος συντακτικός αναλυτής επιστρέφει ένα Abstract Syntax Tree ως λεκτικό.


In [7]:
# parser (έκδοση 1)

# ορισμός προσεταιριστικότητας και προτεραιότητας τελεστών (ο πολλαπλασιασμός και η διαίρεση έχουν μεγαλύτερη προτεραιότητα από την πρόσθεση και την αφαίρεση)
precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'MULTIPLY', 'DIVIDE')
)

# ο πρώτος κανόνας ορίζει και το αρχικό σύμβολο της γραμματικής
# pcalc
def p_calc(p):
    '''
    calc : expression
         | id_assign
         | empty
    '''
    print(p[1])

# expression
def p_expression(p):
    '''
    expression : expression MULTIPLY expression
               | expression DIVIDE expression
               | expression PLUS expression
               | expression MINUS expression
    '''
    p[0] = (p[2], p[1], p[3])

# expression
def p_expression_int_float(p):
    '''
    expression : INT
               | FLOAT
    '''
    p[0] = p[1]

# expression
def p_expression_id(p):
    '''
    expression : ID
    '''
    p[0] = ('id', p[1])

# id_assign
def p_id_assign(p):
    '''
    id_assign : ID EQUALS expression
    '''
    p[0] = ('=', p[1], p[3])

# empty
def p_empty(p):
    '''
    empty : 
    '''
    p[0] = None

def p_error(p):
    print("Syntax error")

parser = yacc.yacc(write_tables=False)

Generating LALR tables


In [8]:
# parsing έκφρασης
s = 'x = 1 + 2'
parser.parse(s)

('=', 'x', ('+', 1, 2))


In [9]:
# parsing έκφρασης με μεταβλητές
s = 'y = 1 + 2 * x'
parser.parse(s)

('=', 'y', ('+', 1, ('*', 2, ('id', 'x'))))


### Συντακτικός αναλυτής (2/2)

Αριθμητική αποτίμηση εκφράσεων.

** BNF γραμματική**

```
calc : expression
     | id_assign
     | empty

expression : expression MULTIPLY expression
           | expression DIVIDE expression
           | expression PLUS expression
           | expression MINUS expression
           | ID
           | INT 
           | FLOAT

id_assign : ID EQUALS expression

empty : 
```

**Συντακτικά οδηγούμενη μετάφραση (Syntax Directed Translation)**

```
Grammar                                         Action
--------------------------------------------    --------------------------------------------
calc0 : expression1                             print expression1.val  
      | id_assign1                              
      | empty                                   

expression : expression MULTIPLY expression     expression0.val = expression1.val * expression3.val
           | expression DIVIDE expression       expression0.val = expression1.val / expression3.val
           | expression PLUS expression         expression0.val = expression1.val + expression3.val
           | expression MINUS expression        expression0.val = expression1.val - expression3.val
           | ID                                 expression0.val = lookup(ID.lexval)
           | INT                                expression0.val = int(INT.lexval)
           | FLOAT                              expression0.val = float(FLOAT.lexval)

id_assign : ID EQUALS expression                store(ID.lexval, expression.val) ; print ID.lexval stored to locals dictionary

empty : 
```


In [10]:
# parser (έκδοση 2)

precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'MULTIPLY', 'DIVIDE')
)

def p_calc(p):
    '''
    calc : expression
         | id_assign
         | empty
    '''
    result = run(p[1])
    if result is not None:
        print(result)

# expression
def p_expression(p):
    '''
    expression : expression MULTIPLY expression
               | expression DIVIDE expression
               | expression PLUS expression
               | expression MINUS expression
    '''
    p[0] = (p[2], p[1], p[3])

# expression
def p_expression_int_float(p):
    '''
    expression : INT
               | FLOAT
    '''
    p[0] = p[1]

# expression
def p_expression_id(p):
    '''
    expression : ID
    '''
    p[0] = ('id', p[1])

# id_assign
def p_id_assign(p):
    '''
    id_assign : ID EQUALS expression
    '''
    p[0] = ('=', p[1], p[3])

# empty
def p_empty(p):
    '''
    empty : 
    '''
    p[0] = None

# error
def p_error(p):
    print("Syntax error")

locals={}
# run
def run(p):
    if type(p) == tuple:
        if p[0] == '+':
            return run(p[1]) + run(p[2])
        elif p[0] == '-':
            return run(p[1]) - run(p[2])
        elif p[0] == '*':
            return run(p[1]) * run(p[2])
        elif p[0] == '/':
            return run(p[1]) / run(p[2])
        elif p[0] == '=':
            locals[p[1]] = run(p[2])
            print(f"Identifier '{p[1]}' set to {locals[p[1]]}")
        elif p[0] == 'id':
            if p[1] not in locals:
                print(f"Unknown identifier {p[1]}")
                sys.exit(-1)
            else:
                return locals[p[1]]
    else:
        return p
    
parser = yacc.yacc(write_tables=False)

Generating LALR tables


In [11]:
s = '1 + 2 * 3'
parser.parse(s)

7


In [12]:
parser.parse('a = 1')
parser.parse('b = 99')
parser.parse('x = 7.5 * a + 8 * b')
parser.parse('x*2')

Identifier 'a' set to 1
Identifier 'b' set to 99
Identifier 'x' set to 799.5
1599.0


In [13]:
parser.parse('x = 1 + k')

Unknown identifier k


SystemExit: -1